<a href="https://colab.research.google.com/github/mnaredla-cloud/GenAI/blob/main/Multi_Layer_Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from google.colab import files
uploaded = files.upload()

words = open('names.txt', 'r').read().splitlines()
words[:10]

Saving names.txt to names.txt


['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [4]:
len(words)

32033

In [5]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [17]:
block_size = 3
X, Y = [], []
for w in words:
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    context = context[1:] + [ix]
X = torch.tensor(X)
Y = torch.tensor(Y)

In [23]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator = g)
W1 = torch.randn((6, 100), generator = g)
b1 = torch.randn(100, generator = g)
W2 = torch.randn((100, 27), generator = g)
b2 = torch.randn(27, generator = g)
parameters = [C, W1, b1, W2, b2]

In [14]:
sum(p.nelement() for p in parameters) #Total no:of parameters in the model

3481

In [25]:
for p in parameters:
  p.requires_grad = True

In [28]:
X.shape

torch.Size([228146, 3])

In [29]:
torch.randint(0,5,(31,))

tensor([2, 3, 1, 0, 2, 1, 1, 3, 3, 4, 0, 1, 0, 2, 0, 0, 2, 3, 0, 1, 4, 1, 2, 2,
        2, 2, 2, 0, 2, 2, 2])

In [32]:
ix = torch.randint(0, X.shape[0], (32,))

In [34]:
C[X[ix]]

tensor([[[ 0.3797, -0.3591],
         [ 0.3797, -0.3591],
         [ 0.3797, -0.3591]],

        [[ 0.3797, -0.3591],
         [-0.0085, -0.3227],
         [ 0.3864,  2.1783]],

        [[ 0.1678, -0.3464],
         [-0.1603, -0.3740],
         [-0.0085, -0.3227]],

        [[ 0.0780, -0.3170],
         [-0.1986, -0.3445],
         [-0.0085, -0.3227]],

        [[ 0.1678, -0.3464],
         [-0.0512, -0.3861],
         [-0.3749, -0.5842]],

        [[-0.1603, -0.3740],
         [ 0.0780, -0.3170],
         [ 0.1678, -0.3464]],

        [[ 0.3797, -0.3591],
         [ 0.3797, -0.3591],
         [ 0.3864,  2.1783]],

        [[ 0.3797, -0.3591],
         [ 0.3797, -0.3591],
         [-0.0057, -0.4284]],

        [[-0.1603, -0.3740],
         [ 0.0780, -0.3170],
         [ 0.5530,  2.3929]],

        [[-0.0057, -0.4284],
         [-0.0057, -0.4284],
         [ 0.1678, -0.3464]],

        [[ 0.1678, -0.3464],
         [-0.1451, -0.1908],
         [-0.1046, -0.2344]],

        [[ 0.3864,  2

In [37]:
for _ in range(10):
  #minibatch
  ix = torch.randint(0, X.shape[0], (32,))

  #forward_pass
  emb = C[X][ix]
  h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
  logits = h @ W2 + b2
  loss = F.cross_entropy(logits, Y[ix])
  print(loss.item())

  #backward pass
  for p in parameters:
    p.grad = None
  loss.backward()

  #update
  for p in parameters:
    p.data += -0.1 * p.grad

2.852262496948242
2.8554089069366455
2.6280906200408936
2.6322951316833496
2.8489797115325928
2.5175983905792236
2.665717124938965
3.1750664710998535
2.6490330696105957
2.446056604385376
